* Find file location on disk
  * Place under Git
  * Create new IHE-SDC Git repo for SDC Object Model Usage Samples
  * Create a folder for SDC XML files
  * Create "Assmblies" folder for imported assemblies
    * Copy in the latest SDC.Schema dll 
    * Copy in "C:\Program Files\Microsoft Visual Studio\2022\Enterprise\Common7\IDE\PublicAssemblies\Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll" //Microsoft.VisualStudio.TestTools.UnitTesting
    * Create Archive folder for junk notebooks

## INIT

In [ ]:
//Import assemblies from files
#r "./Assemblies/SDC_CodeGeneratorTest.dll"  //SDC.Schema namespace
#r "./Assemblies/Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll"  //unit test framework, if we want to use Asserts etc.
//https://github.com/dotnet/interactive/blob/main/docs/nuget-overview.md
#r "nuget:CSharpVitamins.ShortGuid"

using System;
using SDC.Schema;
    using Microsoft.VisualStudio.TestTools.UnitTesting;
    using static CSharpVitamins.ShortGuid;
//
    //using System.IO;
    //using System.Runtime;
    //using Newtonsoft.Json;
    //using System.Data;
    //using System.Diagnostics;
    //using System.Drawing;
    //using System.Linq;
    //using System.Xml; 

### Read in a large SDC file, and then deserialize as XML

In [ ]:
string path = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml";
FormDesignType FD;
BaseType.ResetSdcImport();
//FD.TreeLoadReset(); //runs BaseType.ResetSdcImport();
FD = SDC.Schema.FormDesignType.DeserializeFromXmlPath(path); //Deserialize
display (FD.Nodes.Equals(FD.TopNode.Nodes));
string myXML =  SdcSerializer<FormDesignType>.Serialize(FD); //Serialize
var sGuid = CSharpVitamins.ShortGuid.NewGuid();
display(FD.Nodes.Count());
display(sGuid);
//sGuid
//myXML

### Create iterator to return nodes

In [ ]:
using System.Diagnostics;
#nullable enable
//------------------------------------------------------------------------
////SDC Setup
    BaseType.ResetSdcImport();
    string pathNew = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml";
    var fdNew = SDC.Schema.FormDesignType.DeserializeFromXmlPath(pathNew); //Deserialize
    //---------------------------------------------------------
    FormDesignType.ResetSdcImport();
    string pathOld = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_Copy_sdcFDF.xml";
    var fdOld = SDC.Schema.FormDesignType.DeserializeFromXmlPath(pathOld); //Deserialize
////TIMER SETUP
    Stopwatch.StartNew();
    var timerStartTime  = (float)Stopwatch.GetTimestamp();
    //display(timerStartTime);
//---------------------------------------------------------------------

int nodeCountNew = 0;
int nodeMatchCount = 0;
//Create a new dictionary for the Old template, keyed by name
    var nodesOld = fdOld.Nodes.Values;//this is not guaranteed to be in order, since each node can move within the tree.
    //var nodesOld = fdOld.GetSortedNodesList(); //GetSortedNodesList is not present in the old OM, so we can't use it yet.
    //fdOld.GetNodeNext() //GetNodeNext uses reflection to move next, so it may be slow
    
    var ietDictOld = new Dictionary<string, IdentifiedExtensionType>();  //uses @ID as key
    display("nodesOld.Count: " + nodesOld.Count());

    foreach(BaseType n in nodesOld)
    {
        if((n is not ITopNode) && (n is IdentifiedExtensionType nIET) && (nIET.ID is not null)) 
            ietDictOld.Add(nIET.ID, nIET);
    }

////////////////////////////////////////////////////////////////////////////////////////////
display("ietDictOld: " + ietDictOld.Count());
int unNamedIetNodes = 0;
var addedNodes = new Dictionary<string, BaseType>();

//Iterate the New tree with iET nodes, finding matching Old tree nodes as we progress down the tree
foreach (BaseType nodeNew in NodeIterator(fdNew))
    {
        IdentifiedExtensionType? nodeOld;
        //btPrint(node);
        //Handle old-new node comparison
        if(nodeNew is IdentifiedExtensionType ietNew)  //We can use @ID instead of @name to do the node comparisons
        {
            nodeCountNew++;
            if (ietNew.ID is null) unNamedIetNodes++;
            if (ietNew.ID is not null && ietDictOld.TryGetValue(ietNew.ID, out nodeOld)) 
            {
                nodeMatchCount++;
                //display(nodeOld.name); 
            }
            else
            {
                    if(ietNew.ID is not null) addedNodes.Add(ietNew.ID, ietNew);  //for new node annotations       
                    //could raise event here or start Async Task to add "new" icon to the new SDC tree (fdNew)
            }                       
        }
    }
    
    display("New IET nodes count: " + nodeCountNew);
    display("Node match count: " + nodeMatchCount);
    display("unNamedIetNodes: " + unNamedIetNodes);
    display("addedNodes #: " + addedNodes.Count());

///////////////////////////////////////////////////////////
    int nodeCountOld = 0;
    foreach (BaseType node in NodeIterator(fdOld))
        {
            //btPrint(node);
            //Handle old-new node comparison
            if(node is IdentifiedExtensionType iet)
            {
                nodeCountOld++;
            }
        }
        display("Old IET nodes count: " + nodeCountOld);


//////////////////////  TODO:      COPY NodeIterator and MoveNext to SDC OM     (uses TopNode.ChildNodes, not reflection)      /////////////////////////////
//////////////////////  TODO:      Add iterator pattern to ITopNode, allowing "foreach" on any top level SDC node             ///////////////////////
IEnumerable<BaseType> NodeIterator(BaseType? n)
    {
        int nodeCounter = 0;
        while (n is not null)
        {
            n = MoveNext(n, ref nodeCounter);
            //display("nodecounter: " + nodeCounter);
            if (n is not null)
                yield return n;
            else yield break;
        }
        yield break;
    }
/////////////////////////////////////////////////

BaseType? MoveNext(BaseType n, ref int nodeCounter)
{
    Dictionary<Guid, List<BaseType>> cn = n.TopNode.ChildNodes;
    List<BaseType>? childList;
    BaseType? nextNode;
    
    n.order = nodeCounter;  //almost instananeous
    Assert.IsTrue(n.ObjectID == nodeCounter);//very fast
    nodeCounter++;
    //if n has child nodes, the next node is the first child node of n.
    if (cn.TryGetValue(n.ObjectGUID, out childList))
    {
        nextNode = childList[0];
        if (nextNode is not null) return nextNode;
    }
    //Notes:
        //n has no child nodes here, so walk up the tree to the parent node.
        //When we walk back up the object graph, prevPar is the original starting node (deeper in the tree),
        //while par is the parent of prevPar.  par is more superficial in the tree, and closer to the top node
        //We then check the childList of par, to see if prePar can be found in that childList
        //If prevPar is in childList, then try to retrieve the next node in childList
        //If we can't get the nextNode from childList, then move up to one higher parent level
    var prevPar = n;  
    var par = prevPar.ParentNode;

    while (par is not null)
    {
        if (cn.TryGetValue(par.ObjectGUID, out childList))
        {
            var index = childList.IndexOf(prevPar!);
            if (index < childList.Count - 1)
            {
                nextNode = childList[index + 1];
                if (nextNode is not null) return nextNode;
            }
            //the next node is not located yet, so walk up to a previous ancestor and try again,
            //looking in that ancestors childList for a nextNode candidate
            prevPar = par; 
            par = prevPar.ParentNode;
        }
    }
    return null;
}
/////////////////////////Small method to output node info///////////////////////////////
void btPrint(BaseType n)
    {
        string content;
        if (n is not null)
        {
            if (n is DisplayedType) content = ": title: " + (n as DisplayedType)?.title;
            else if (n is PropertyType) content = ": " + (n as PropertyType)?.propName + ": " + (n as PropertyType)?.val;
            else content = "";

            display(n.ObjectID.ToString().PadLeft(4) + ": " + n.order.ToString().PadLeft(4) + ": " + (n.name ?? "").PadRight(20) + ": " + (n.ElementName ?? "").PadRight(25) + content);
        }
    }
//display(Stopwatch.GetTimestamp());
display("Elapsed time (sec) = " + ((Stopwatch.GetTimestamp() - timerStartTime) / Stopwatch.Frequency).ToString());

In [ ]:
////TIMER SETUP
Stopwatch.StartNew();
var timerStartTime  = (float)Stopwatch.GetTimestamp();
// string path = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml";
// var fdNew = SDC.Schema.FormDesignType.DeserializeFromXmlPath(path); //Deserialize

int i=0;
display(fdNew.Nodes.Values.Count());
//foreach (BaseType n in NodeIterator(fdNew))
foreach (BaseType n in fdNew.Nodes.Values)
{    i++;   }
display("Elapsed time (sec) = " + ((Stopwatch.GetTimestamp() - timerStartTime) / Stopwatch.Frequency).ToString());
display(i);
//--------------------------------------------------------------
Stopwatch.StartNew();
timerStartTime  = (float)Stopwatch.GetTimestamp();
i=0;
foreach (BaseType n in fdNew.GetSortedNodesList()) i++;


display("Elapsed time Sorted Nodes (sec) = " + ((Stopwatch.GetTimestamp() - timerStartTime) / Stopwatch.Frequency).ToString());
display(i);

2431

Elapsed time (sec) = 0

2431

Elapsed time Sorted Nodes (sec) = 0

2431

### Find SDC node changes compared to the reference template
* Create 2 FormDesign OMs, "fdOld" and "fdNew"
* Iterate each node in fdNew, and locate matcching node in fdOld by ID, SGuid and/or name
  * For each fdNew node, determine if it's 
    * new, 
    * has a different parent node, 
    * previous sib has changed.
  * Add the changed node's ID/name/sGUID and change type to a dictionary
* Compare each SDC property in fdNew with the matching fdOld node
*  

In [ ]:
// var a1 = new { A = 1, B = 2, C = 3, D = 4, E = 5 };
// var a2 = a1 with { E = 10 }; 
// Console.WriteLine (a2);      // { A = 1, B = 2, C = 3, D = 4, E = 10 }
using System.Linq;
#nullable enable
//Should we add the affected subnode (propNode) to the struct?
//public readonly record struct nDiff (BaseType nOld, BaseType propNode, string ChangedPropName , string? ChangedPropVal);
public readonly record struct nDiff (BaseType nOld, string ChangedPropName , string? ChangedPropVal);


    bool QuestionChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = RepeatTypeChanged(nOld, nNew, nChanges);

        //QuestionItemBaseType
        if (nOld.readOnly != nNew.readOnly) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.readOnly), nOld.readOnly.ToString()));}
        if (nOld.changedData != nNew.changedData) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.changedData), nOld.changedData.ToString()));}
        if (nOld.newData != nNew.newData) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.newData), nOld.newData.ToString()));}
        if (nOld.GetQuestionSubtype() != nNew.GetQuestionSubtype()) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "QuestionSubtype", nOld.GetQuestionSubtype().ToString()));}
        
        if (!SpecialPropertyChanged(nOld, nNew, nChanges)) changed = false;

        if (nOld.GetQuestionSubtype() == QuestionEnum.QuestionFill)
        {
            //Look for some special QR properties on sub-nodes, like datatype, minInclusive, maxInclusive, fractionDigits, val, quantEnum, etc.
            //Surface any changes, so they appear in this Question node
            //Also handle TextAfterResponse, Response Units

            //Retrieve DataTypes_DEtype object (Response Element)
            DataTypes_DEType? respOld = nOld.ResponseField_Item?.Response;
            DataTypes_DEType? respNew = nNew.ResponseField_Item?.Response;

            string? respTextAfterOld = nOld.ResponseField_Item?.TextAfterResponse?.val;
            string? respTextAfterNew = nNew.ResponseField_Item?.TextAfterResponse?.val;

            string? respUnitsOld = nOld.ResponseField_Item?.ResponseUnits?.val;
            string? respUnitsNew = nNew.ResponseField_Item?.ResponseUnits?.val;

            string? unitSysOld = nOld.ResponseField_Item?.ResponseUnits?.unitSystem;
            string? unitSysNew = nNew.ResponseField_Item?.ResponseUnits?.unitSystem;

            string? dtOldName = respOld?.DataTypeDE_Item?.NodeType.ToString();
            string? dtNewName = respNew?.DataTypeDE_Item?.NodeType.ToString();

            BaseType? dtOldNode = respOld?.DataTypeDE_Item;
            BaseType? dtNewNode = respNew?.DataTypeDE_Item;

            Type? dtOld = dtOldNode?.GetType();
            Type? dtNew = dtNewNode?.GetType();
            //Need to check that dtOld=dtNew, and handle gracefully



            System.Reflection.FieldInfo[]? fieldsOld = dtOld?.GetFields(); //TODO: only retrieve serializable fields via attributes?
            System.Reflection.FieldInfo[]? fieldsNew = dtNew?.GetFields(); //TODO: only retrieve serializable fields via attributes?

            dynamic? maxIncOld = fieldsOld?.Where(f=>f.Name=="maxInclusive")?.First()?.GetValue(dtOldNode); //this may not work, since we don't know the type at compile time
            dynamic? maxIncNew = fieldsNew?.Where(f=>f.Name=="maxInclusive")?.First()?.GetValue(dtNewNode); //Conside dynamic type or converting results to string

            dynamic? minIncOld = fieldsOld?.Where(f=>f.Name=="minInclusive")?.First()?.GetValue(dtOldNode);
            dynamic? minIncNew = fieldsNew?.Where(f=>f.Name=="minInclusive")?.First()?.GetValue(dtNewNode);

            dynamic? maxExOld = fieldsOld?.Where(f=>f.Name=="maxExclusive")?.First()?.GetValue(dtOldNode); //this may not work, since we don't know the type at compile time
            dynamic? maxExNew = fieldsNew?.Where(f=>f.Name=="maxExclusive")?.First()?.GetValue(dtNewNode);

            dynamic? minExOld = fieldsOld?.Where(f=>f.Name=="minExclusive")?.First()?.GetValue(dtOldNode);
            dynamic? minExNew = fieldsNew?.Where(f=>f.Name=="minExclusive")?.First()?.GetValue(dtNewNode);

            byte? fracDigOld = fieldsOld?.Where(f=>f.Name=="fractionDigits")?.First()?.GetValue(dtOldNode) as byte?;
            byte? fracDigNew = fieldsNew?.Where(f=>f.Name=="fractionDigits")?.First()?.GetValue(dtNewNode) as byte?;

            dynamic? valOld = fieldsOld?.Where(f=>f.Name=="val")?.First()?.GetValue(dtOldNode);
            dynamic? valNew = fieldsNew?.Where(f=>f.Name=="val")?.First()?.GetValue(dtNewNode);

            byte? totDigOld = fieldsOld?.Where(f=>f.Name=="totalDigits")?.First()?.GetValue(dtOldNode) as byte?;
            byte? totDigNew = fieldsNew?.Where(f=>f.Name=="totalDigits")?.First().GetValue(dtNewNode) as byte?;

            long? minLenOld = fieldsOld?.Where(f=>f.Name=="minLength")?.First()?.GetValue(dtOldNode) as byte?;
            long? minLenNew = fieldsNew?.Where(f=>f.Name=="minLength")?.First()?.GetValue(dtNewNode) as byte?;

            long? maxLenOld = fieldsOld?.Where(f=>f.Name=="maxLength")?.First()?.GetValue(dtOldNode) as byte?;
            long? maxLenNew = fieldsNew?.Where(f=>f.Name=="maxLength")?.First()?.GetValue(dtNewNode) as byte?;

            string? maskOld = fieldsOld?.Where(f=>f.Name=="mask")?.First()?.GetValue(dtOldNode) as string;
            string? maskNew = fieldsNew?.Where(f=>f.Name=="mask")?.First()?.GetValue(dtNewNode) as string;

            string? patternOld = fieldsOld?.Where(f=>f.Name=="pattern")?.First()?.GetValue(dtOldNode) as string;
            string? patternNew = fieldsNew?.Where(f=>f.Name=="pattern")?.First()?.GetValue(dtNewNode) as string;

            string? quantEnumOld = fieldsOld?.Where(f=>f.Name=="quantEnum")?.First()?.GetValue(dtOldNode) as string;
            string? quantEnumNew = fieldsNew?.Where(f=>f.Name=="quantEnum")?.First()?.GetValue(dtNewNode) as string;

            //TODO:
                //allowGT
                //allowGTE
                //allowLT
                //allowLTE
                //allowAPPROX

        }
        if (nOld.GetQuestionSubtype() == QuestionEnum.QuestionSingleOrMultiple)
        {
            //Surface minSelections, maxSelections, numCols... changes to this Question node
        }

        return changed;
    }

    bool SpecialPropertyChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;

        if(!ReportTextPropertyChanged(nOld, nNew, nChanges)) changed = true;
        if(!AltTextPropertyChanged(nOld, nNew, nChanges)) changed = true;

        return changed;
    }
    bool PropertyChanged(PropertyType nOld, PropertyType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
        if(BaseTypeChanged(nOld, nNew, nChanges)) changed = true;
        if (nOld.propName != nNew.propName) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.propName), nOld.propName));}
        if (nOld.propClass != nNew.propClass) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.propClass), nOld.propClass));}
        if (nOld.val != nNew.val) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.val), nOld.val));}
        
        return changed;
    }
    bool PropertyRepeatsChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
        {
            bool changed = false;
            //Group all Properties by propName, for both nOld and nNew,
            //so that we can determine if some property nodes occur more than once, or if the counts differ between nOld and nNew
            //We are using nNew as teh basis for comparison, such that any property nodes that were removed from nOld will not show up here
            //-------------------------
            var pNew =
            from p in nNew.GetChildList().OfType<PropertyType>().ToList()
            group p by p.propName into propsByPropName
            select propsByPropName;
            //------------------------
            var pOld =
            from p in nNew.GetChildList().OfType<PropertyType>().ToList()
            group p by p.propName into propsByPropName
            select propsByPropName;
            //------------------------

            foreach (IGrouping<string, PropertyType> propsByPropNameNew in pNew)
            {
                //iterate through the propName-grouped Properties in pNew
                foreach (IGrouping<string, PropertyType> propsByPropNameOld in pOld.Where(p=> propsByPropNameNew.Key ==p.Key))
                {//find propName-grouped Properties in pOld (i.e., the current IGrouping named propsByPropNameOld) where the propName matches the current IGrouping from pNew (propsByPropNameNew)
                    //i.e., where propsByPropNameNew.Key ==p.Key

                    var newCount = propsByPropNameNew.Count();
                    var oldCount = propsByPropNameNew.Count();
                    //If the counts for the current propName differ in pOld and pNew, flag it as a change:
                    if (newCount != oldCount) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "Counts_OldPropName_NewPropName", oldCount.ToString() + " - " +  newCount.ToString()));}
                }
                //Search for propNames in pOld that are not found in pNew
            }            
            return changed;
        }
    bool ReportTextPropertyChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
        //reportText
        List<PropertyType>? newReportText = nNew.GetChildList().OfType<PropertyType>()?.Where(p=>p.propName == "reportText")?.ToList();
        List<PropertyType>? oldReportText = nOld.GetChildList().OfType<PropertyType>()?.Where(p=>p.propName == "reportText")?.ToList();
        if (newReportText?[0] !=oldReportText?[0]) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "reportText", oldReportText?[0]?.ToString()));}  

        return changed;
    }
    bool AltTextPropertyChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
        //reportText
        List<PropertyType>? newAltText = nNew.GetChildList().OfType<PropertyType>()?.Where(p=>p.propName == "altText")?.ToList();
        List<PropertyType>? oldAltText = nOld.GetChildList().OfType<PropertyType>()?.Where(p=>p.propName == "altText")?.ToList();
        if (newAltText?[0] !=oldAltText?[0]) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "altText", oldAltText?[0]?.ToString()));}
        //(oldAltText[0] is null)?"":oldAltText[0].val)
        return changed;
    }
    bool FormDesignHeaderPropertyChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = BaseTypeChanged(nOld, nNew, nChanges);
        //Assess FormDesign attributes
        //Assess Properties         

        return changed;
    }
    bool LIRDatatypeChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = BaseTypeChanged(nOld, nNew, nChanges);
        //Assess the datatype element
        //Assess Properties         

        return changed;
    }
    bool RepeatTypeChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = DisplayedTypeChanged(nOld, nNew, nChanges);
                  
        //RepeatType
        if (nOld.minCard != nNew.minCard) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.minCard), nOld.minCard.ToString()));}
        if (nOld.maxCard != nNew.maxCard) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.maxCard), nOld.maxCard.ToString()));}
        if (nOld.repeat != nNew.repeat) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.repeat), nOld.repeat.ToString()));}
        if (nOld.instanceGUID != nNew.instanceGUID) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.instanceGUID), nOld.instanceGUID));}
        if (nOld.parentGUID != nNew.parentGUID) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.parentGUID), nOld.parentGUID));}

        return changed;
    }
    bool DisplayedTypeChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = IdentifiedExtensionTypeChanged(nOld, nNew, nChanges);

        //DisplayedType
        if (nOld.title != nNew.title) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.title), nOld.title));}
        if (nOld.enabled != nNew.enabled) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.enabled), nOld.enabled.ToString()));}
        if (nOld.visible != nNew.visible) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.visible), nOld.visible.ToString()));}
        if (nOld.mustImplement != nNew.mustImplement) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.mustImplement), nOld.mustImplement.ToString()));}
        if (nOld.showInReport != nNew.showInReport) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.showInReport), nOld.showInReport.ToString()));}

        return changed;
    }
    bool IdentifiedExtensionTypeChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = BaseTypeChanged(nOld, nNew, nChanges);
        
        //IET
        if (nOld.ID != nNew.ID) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.ID), nOld.ID));}
        if (nOld.baseURI != nNew.baseURI) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.baseURI), nOld.baseURI));}

        return changed;
    }
    bool BaseTypeChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
        
        if (nOld.name != nNew.name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.name), nOld.name));}
        if (nOld.type != nNew.type) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.type), nOld.type));}
        if (nOld.styleClass != nNew.styleClass) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.styleClass), nOld.styleClass));}
        if (nOld.sGuid != nNew.sGuid) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.sGuid), nOld.sGuid));}
        if (nOld.order != nNew.order) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.order), nOld.order.ToString()));}

        if (nOld.ParentNode.name != nNew.ParentNode.name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "ParentNode.name", nOld.ParentNode.name));}
        if (nOld.ParentIETypeNode.name != nNew.ParentIETypeNode.name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "ParentIETypeNode.name", nOld.ParentIETypeNode.name));}
        if (nOld.ParentIETypeID != nNew.ParentNode.ParentIETypeID) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.ParentIETypeID), nOld.ParentIETypeID));}
        
        //Check previous sibling match, if present
        BaseType? prevSibOld = nOld.GetNodePreviousSib();
        BaseType? prevSibNew = nNew.GetNodePreviousSib();
        if(prevSibOld.name != prevSibNew.name)
        {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "PreviousSib.name", prevSibOld?.name??""));}

        //Check node type
        if (nOld.GetType().Name != nNew.GetType().Name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "GetType.Name", nOld.GetType().Name));}
        //Check parent node type
        if (nOld.ParentNode.GetType().Name != nNew.ParentNode.GetType().Name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "ParentNode.GetType.Name", nOld.ParentNode.ParentNode.GetType().Name));}
        //Check all Properties, Comments, Extensions here?

        if (changed) return false;            

        return true;
    }



In [ ]:
var bt = new FormDesignType();

if(null == null) display(true);

int i = 123;
object o = new object();
object o2;
display(o.GetType().Name);
o= i;

display(o.ToString());
display(o.GetType().Name);
display(o2.ToString());

## Create methods for:
 - Add sGUID to every node (if not already present)
 - Add name attribute to every node (if not present)
 - Save SDC XML to disk, with option to overwrite or save as new file
 - Add a new status flag like REL+ to indicate that sGuids are present in the file
 - Construct the file name from FormDesign header data

### Assign sGuids to Nodes

In [ ]:
static void AssignsGuidtoNodes(ITopNode tn, bool overwritesGuid = false)
{
    foreach(BaseType n in tn.Nodes.Values)
    {
        if (overwritesGuid || n.sGuid is null) n.sGuid = (CSharpVitamins.ShortGuid.NewGuid());
    }
    //save file here
}


### Assign Name to Nodes

In [ ]:
# nullable enable
static void AssignsNametoNodes(ITopNode tn, bool overwriteNames = false)
{
    int i = 0;
    foreach(BaseType n in tn.GetSortedNodesList())
    {   
        display("----------------------------");
        display ("Node: " + n.name + "\n Type: " + n.GetType().Name);
        display("   ParentNode Type is: "+ (n.ParentNode?.GetType().Name));
        display("   Parent IET is: "+ n.ParentIETypeNode?.ID);      
        IdentifiedExtensionType? parIET = ParentIETypeNode(n);
        display("   Parent IET (local method) is: " + parIET?.ID + ", name:" + parIET?.name);  
        display("   New name: " + CreateName(n));
        i++;
        if(i>40) goto end;
    }
    end:;
    //save file here
}

static string CreateName(BaseType bt)
{
    string shortID = "";
    string prefix = "";
    string shortName = "";
    //int ietCounter = 0;

    if (bt is IdentifiedExtensionType iet)
    {
        //ietCounter = 0;
        shortID = TruncateID(iet);
        prefix = bt.ElementPrefix;
        //TODO: shortName is commented out until we have a chance to QA the names
        //bt.BaseName = ((string)drFormDesign["ShortName"]).Replace(" ", "_");
        //shortName = bt.BaseName;
        //if (shortName.Length > 0) shortName += "_";  //I don't like 2 underscores next to each other.
        if (prefix.Length > 0) prefix += "_";
        return prefix + shortName + shortID;
    }
        else
    {
        //ietCounter++;
        //var ietPar = bt.ParentIETypeNode;
        var ietPar = ParentIETypeNode(bt);
        if(ietPar is null) 
        {
            display($"node {bt.name} has no parent IET node");
            return "";
        }
        shortID = TruncateID(ietPar) + "_";
        shortName = bt.BaseName.Replace(" ", "_");
        prefix = bt.ElementPrefix;
        if (shortName.Length > 0) shortName += "_";
        if (prefix.Length > 0) prefix += "_";

        if (ietPar.ID.EndsWith("_Body"))
        {
            shortID = "Body";
            return prefix + shortName + shortID;
        }
        return prefix + shortName + shortID + GetIetCounter(bt).ToString();  //SubIETcounter needs to be public, or else, move these fucntions into the SDC OM
                    //or create a fucntion to calculate this on the fly, by walking up to the nearest IET ancestor; this technique is brittle during the editing process, except right before release.
                    //it may be better to rely on sGuids for stable node identifiers, and only assign names before release when the node order is stable.
    }
}

static int GetIetCounter(BaseType bt)
{
    if(bt is IdentifiedExtensionType iet) return 0;

    int ietCounter = 0;
    BaseType prev = bt.GetNodePrevious();
    while (prev is not null)
    {
        ietCounter++;
        if(ietCounter>10) 
        {
            display(prev.name);
        throw new Exception("ietCounter>10");
        }
        //display("ietCounter: " + ietCounter);
        if(prev is IdentifiedExtensionType) return ietCounter;
    }
    return 0;
}

static string TruncateID(IdentifiedExtensionType iet)
{
    //display("iet.ID " + iet?.ID);
    string shortID = iet.ID;
    int i = 0;
    if (!string.IsNullOrWhiteSpace(iet.ID))
    {
        i = iet.ID.ToString().IndexOf(".100004300");
        if (i > 0)
            shortID = iet.ID.ToString().Substring(0, i);
    }
    return shortID;
}

static IdentifiedExtensionType ParentIETypeNode(BaseType bt)
{
        BaseType? node = bt;
        BaseType? outParentNode;
        do
        {
            node.TopNode.ParentNodes.TryGetValue(node.ObjectGUID, out outParentNode);

            if (outParentNode != null && outParentNode is IdentifiedExtensionType)
                return (IdentifiedExtensionType)outParentNode;
            node = node?.ParentNode;
        } while (node != null);
        return null!;

}
AssignsNametoNodes(FD);

In [ ]:
display (FD.Nodes.Equals(FD.TopNode.Nodes));
display(FD.Nodes.Values.Count());
display(FD.Nodes.Values.OfType<PropertyType>().Count());
var p = FD.Nodes.Values.OfType<PropertyType>()?.Where(n=> n.propName == "Copyright")?.FirstOrDefault();
display(p?.propName);
display(FD.TopNode.Nodes.Count());
display(FD.TopNode.Nodes.Values.OfType<PropertyType>().Count());
p = FD.TopNode.Nodes.Values.OfType<PropertyType>().Where(n=> n.propName == "Copyright")?.FirstOrDefault();
display(p?.propName);

In [ ]:
// display(typeof(FormDesignType) is IdentifiedExtensionType);
// display(FD.GetType() is IdentifiedExtensionType);
// display(FD is IdentifiedExtensionType);
// FD.GetType().IsSubclassOf(typeof(IdentifiedExtensionType))

display(FD.Nodes.Count());
int i = 0;
foreach (BaseType n in FD.GetSortedNodesList())
{
    
    if (i > 100) goto end;
    display(n.name);
    display(n.ParentIETypeNode?.ID);
    display(ParentIETypeNode(n)?.ID);
    i++;
}
end:;
i





foreach (BaseType n in fdNew.GetSortedNodesList()) 
{
    
}